In [ ]:
# ЧЕРНОВОЙ ВАРИАНТ добавления ссылок на сайт
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf /content/drive/MyDrive/KIA_TEST/HOTFIX_MODELS_LINKS
!mkdir -p /content/drive/MyDrive/KIA_TEST/HOTFIX_MODELS_LINKS
!git clone -b hotfix_models_links https://github.com/musicnova/KIA-GPT0.git HOTFIX_MODELS_LINKS
!cp -r /content/HOTFIX_MODELS_LINKS/knowledge/ /content/drive/MyDrive/KIA_TEST/HOTFIX_MODELS_LINKS/knowledge
!rm -r /content/HOTFIX_MODELS_LINKS

Cloning into 'HOTFIX_MODELS_LINKS'...
remote: Enumerating objects: 2560, done.
remote: Counting objects: 100% (973/973), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 2560 (delta 628), reused 820 (delta 541), pack-reused 1587
Receiving objects: 100% (2560/2560), 64.97 MiB | 29.05 MiB/s, done.
Resolving deltas: 100% (1472/1472), done.


In [ ]:
!pip install tiktoken==0.4.0 langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [124]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import NLTKTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.schema.document import Document

with open('/content/drive/MyDrive/KIA_TEST/HOTFIX_MODELS_LINKS/knowledge/database.md', 'r', encoding='utf-8') as file:
    text = file.read().replace('\r', '')

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
]

def meta_to_str(m):
    return '\t'.join([str(m["Header 1"]) if "Header 1" in m else "None"
        ,str(m["Header 2"]) if "Header 2" in m else "None"
        ,str(m["Header 3"]) if "Header 3" in m else "None"
        ,str(m["Header 4"]) if "Header 4" in m else "None"
        ,str(m["Header 5"]) if "Header 5" in m else "None"])

def str_to_meta(s):
    a = s.split('\t')
    return {"Header 1": a[0] if a[0] != 'None' else None
            , "Header 2": a[1] if a[1] != 'None' else None
            , "Header 3": a[2] if a[2] != 'None' else None
            , "Header 4": a[3] if a[3] != 'None' else None
            , "Header 5": a[4] if a[4] != 'None' else None}

# Markdown-level splits
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(text)
md_header_splits_dict = {}
for md_header_split in md_header_splits:
    md_header_splits_key = meta_to_str(md_header_split.metadata)
    if md_header_splits_key in md_header_splits_dict:
        md_header_splits_dict[md_header_splits_key].append(md_header_split.page_content)
    else:
        md_header_splits_dict[md_header_splits_key] = [md_header_split.page_content]

# Character-level splits
char_splitter = CharacterTextSplitter(separator='\r\n', chunk_size=4096, chunk_overlap=0)
char_splits_dict = {}
for md_header_splits_key in md_header_splits_dict:
    for md_header_splits_value in md_header_splits_dict[md_header_splits_key]:
        char_splits = char_splitter.split_text(md_header_splits_value)
        for char_split in char_splits:
            if md_header_splits_key in char_splits_dict:
                char_splits_dict[md_header_splits_key].append(char_split)
            else:
                char_splits_dict[md_header_splits_key] = [char_split]

# Recursive Character-level splits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=0)
text_splits_dict = {}
for char_splits_key in char_splits_dict:
    for char_splits_value in char_splits_dict[char_splits_key]:
        text_splits = text_splitter.split_text(char_splits_value)
        for text_split in text_splits:
            if char_splits_key in text_splits_dict:
                text_splits_dict[char_splits_key].append(text_split)
            else:
                text_splits_dict[char_splits_key] = [text_split]

# Splits
splits = []
for text_splits_key in text_splits_dict:
    for text_splits_value in text_splits_dict[text_splits_key]:
        splits.append(Document(page_content=text_splits_value, metadata=str_to_meta(text_splits_key)))
print(len(splits))

2080


In [125]:
new_splits = []

def get_words(text, random_seed):
    import random
    import re
    random.seed(random_seed)
    words = re.findall(r'\b\w+\b', text)
    start = random.randint(0, len(words) - 6)
    selected_words = [words[start + i] for i in range(0, 6)]
    return selected_words

def duckduckgo(words):
    import random
    import time
    import requests
    from bs4 import BeautifulSoup
    time.sleep(random.randint(0, 2))
    query = 'site:kia.ru ' + words
    url = f"https://duckduckgo.com/html/?q={query}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.select('.result__a')
    if results:
        first_result = results[0]['href']
        return first_result
    else:
        return None


def googlego(words, your_engine_id, your_api_key):
    # README https://help.elfsight.com/article/331-how-to-get-search-engine-id
    # README https://stackoverflow.com/a/11100863
    # README https://developers.google.com/custom-search/v1/introduction
    import random
    import time
    from googleapiclient.discovery import build
    time.sleep(random.randint(0, 2))
    query = 'site:kia.ru ' + words
    cse_id = your_engine_id  # Замените на ваш идентификатор движка поиска
    api_key = your_api_key  # Замените на ваш API-ключ
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id, num=1).execute()
    if 'items' in res:
        return res['items'][0]['link']
    else:
        return None


for split in splits:
    if split.metadata['Header 1'].strip() != "models???":
        new_splits.append(split)
    else:
        print(str(split.metadata), '\n')
        print(split.page_content, '\n')
        counts = {}
        max_count = 0
        best_url = None
        for seed in range(0, 5):
           words = ' '.join(get_words(split.page_content, seed))
           # url = googlego(words, 'ID', 'API_KEY')
           url = duckduckgo(words)
           if url is not None:
               print(words, ' -> ', url)
               if url in counts:
                   counts[url] += 1
               else:
                   counts[url] = 1
               if counts[url] > max_count:
                   max_count = counts[url]
                   best_url = url
        print("=====", '\n')
        if best_url is not None and split.page_content.find(best_url) < 0:
            new_page_content=split.page_content + "\n" + str(best_url) + "\n"
            print(best_url, '\n')
            new_splits.append(Document(page_content=new_page_content, metadata=split.metadata))
        else:
            new_splits.append(Document(page_content=split.page_content, metadata=split.metadata))
        print("=====", '\n')

In [126]:
import binascii

new_text_splits_dict = {}
for new_split in new_splits:
    new_text_splits_key = meta_to_str(new_split.metadata)
    if new_text_splits_key in new_text_splits_dict:
        new_text_splits_dict[new_text_splits_key].append(new_split.page_content)
    else:
        new_text_splits_dict[new_text_splits_key] = [new_split.page_content]

for cur in range(0, len(new_splits)):
    new_split = new_splits[cur]
    new_text_splits_key = meta_to_str(new_split.metadata)
    new_text_splits_index = new_text_splits_dict[new_text_splits_key].index(new_split.page_content)
    if len(new_text_splits_dict[new_text_splits_key]) > 1:
        new_split.metadata['Header 5'] = ','.join([str(binascii.crc32(new_splits[cur + new_text_splits_index - i].page_content.encode('utf8'))) if i != new_text_splits_index else "..." for i in range(0, len(new_text_splits_dict[new_text_splits_key]))])

with open("/content/drive/MyDrive/KIA_TEST/HOTFIX_MODELS_LINKS/knowledge/new_database.md", "w") as f:
    header_1 = None
    header_2 = None
    header_3 = None
    header_4 = None
    header_5 = None
    for new_split in new_splits:
        new_header_1 = new_split.metadata['Header 1']
        new_header_2 = new_split.metadata['Header 2']
        new_header_3 = new_split.metadata['Header 3']
        new_header_4 = new_split.metadata['Header 4']
        new_header_5 = new_split.metadata['Header 5']
        if new_header_1 is not None and header_1 != new_header_1:
            f.write('# ' + new_header_1 + '\n')
            header_1 = new_header_1
        if new_header_2 is not None and header_2 != new_header_2:
            f.write('## ' + new_header_2 + '\n')
            header_2 = new_header_2
        if new_header_3 is not None and header_3 != new_header_3:
            f.write('### ' + new_header_3 + '\n')
            header_3 = new_header_3
        if new_header_4 is not None and header_4 != new_header_4:
            f.write('#### ' + new_header_4 + '\n')
            header_4 = new_header_4
        if new_header_5 is not None and header_5 != new_header_5:
            f.write('##### ' + new_header_5 + '\n')
            header_5 = new_header_5
        f.write(new_split.page_content + '\n')

    print("OK")

OK
